In [ ]:
from qsc import Qsc
import numpy as np
from matplotlib import pyplot as plt
#stel = Qsc(rc=[1, 0.09], zs=[0, -0.09], nfp=2, etabar=0.95, I2=0.9, order='r2', B2c=-0.7, p2=-600000.)
#stel = Qsc(rc=[1, 0.], zs=[0, ], nfp=3, etabar=0.95, I2=0., order='r1')
#stel = Qsc.from_paper("2022 QH nfp7")
stel = Qsc.from_paper("r1 section 5.1")

In [ ]:
#print(stel.iota) # Rotational transform on-axis for this configuration

#print(stel.d2_volume_d_psi2) # Magnetic well V''(psi)
#print(stel.DMerc_times_r2) # Mercier criterion parameter DMerc multiplied by r^2
#print(stel.min_L_grad_B) # Scale length associated with the grad grad B tensor
#print(stel.grad_grad_B_inverse_scale_length) # Scale length associated with the grad grad B tensor
stel.plot_boundary(r=0.2) # Plot the flux surface shape at the default radius r = 0.1
stel.plot() 

In [ ]:
###export axis data to netcdf format.

import netCDF4 as nc
import os
#from qsc.Frenet_to_Frenet_NB import Frenet_to_Frenet_NB

NFP=stel.nfp
n_max=stel.nfourier+1

nzeta = 2*(n_max+1)
nzetaFull = nzeta*NFP
zeta = np.linspace(0,2*np.pi,nzetaFull,endpoint=False)
zeta = zeta+0.5*zeta[1]  # half grid!

xyz = np.zeros((3,nzetaFull))

r = sum([stel.rc[i]*np.cos(i*stel.nfp*zeta)+stel.rs[i]*np.sin(i*stel.nfp*zeta)  for i in range(len(stel.rc))])
z = sum([stel.zc[i]*np.cos(i*stel.nfp*zeta)+stel.zs[i]*np.sin(i*stel.nfp*zeta)  for i in range(len(stel.zs))])
xyz[0,:]= r*np.cos(zeta)
xyz[1,:]= r*np.sin(zeta)
xyz[2,:]= z

rp = sum([i*stel.nfp*(-stel.rc[i]*np.sin(i*stel.nfp*zeta)+stel.rs[i]*np.cos(i*stel.nfp*zeta))  for i in range(len(stel.rc))])
zp = sum([i*stel.nfp*(-stel.zc[i]*np.sin(i*stel.nfp*zeta)+stel.zs[i]*np.cos(i*stel.nfp*zeta))  for i in range(len(stel.zs))])

xyzp = np.zeros((3,nzetaFull))
xyzp[0,:]=-r*np.sin(zeta)+rp*np.cos(zeta)
xyzp[1,:]= r*np.cos(zeta)+rp*np.sin(zeta)
xyzp[2,:]= zp

rpp = sum([-(i*stel.nfp)**2*(stel.rc[i]*np.cos(i*stel.nfp*zeta)+stel.rs[i]*np.sin(i*stel.nfp*zeta))  for i in range(len(stel.rc))])
zpp = sum([-(i*stel.nfp)**2*(stel.zc[i]*np.cos(i*stel.nfp*zeta)+stel.zs[i]*np.sin(i*stel.nfp*zeta))  for i in range(len(stel.zs))])

xyzpp = np.zeros((3,nzetaFull))
xyzpp[0,:]=-r*np.cos(zeta)-2*rp*np.sin(zeta)+rpp*np.cos(zeta)
xyzpp[1,:]=-r*np.sin(zeta)+2*rp*np.cos(zeta)+rpp*np.sin(zeta)
xyzpp[2,:]= zpp

Txyz = np.zeros((3,nzetaFull))
Nxyz = np.zeros((3,nzetaFull))
Bxyz = np.zeros((3,nzetaFull))


lp=np.linalg.norm(xyzp,axis=0)
Txyz[0,:]=xyzp[0,:]/lp
Txyz[1,:]=xyzp[1,:]/lp
Txyz[2,:]=xyzp[2,:]/lp
Bxyz=np.cross(xyzp,xyzpp,axis=0)
absB=np.linalg.norm(Bxyz,axis=0)
Bxyz[0,:]=Bxyz[0,:]/absB
Bxyz[1,:]=Bxyz[1,:]/absB
Bxyz[2,:]=Bxyz[2,:]/absB

Nxyz=np.cross(Bxyz,Txyz,axis=0)

lasym=stel.lasym


fn ="axisNB_2022_QH_nfp7"


os.system("rm -f "+fn+".nc")
ncfile = nc.Dataset(fn+'.nc', 'w') 
vec_dim = ncfile.createDimension('vec',3)
nzeta_dim = ncfile.createDimension('nzeta_axis',nzeta)
nzetaFull_dim = ncfile.createDimension('nzetaFull_axis',nzetaFull)
version=300
for ivar,ival in zip(["NFP","axis/nzeta","VERSION"],["NFP","nzeta","version"]):
    exec(ival+'_var = ncfile.createVariable("'+ivar+'","i8")')
    exec(ival+'_var.assignValue('+ival+')')

zeta_var = ncfile.createVariable("axis/zeta(:)","double",("nzeta_axis"))
zeta_var[:] = zeta[0:nzeta]
for vecvar,vecval in zip(["axis/xyz","axis/Nxyz","axis/Bxyz"],["xyz","Nxyz","Bxyz"]):
    exec(vecval+'_var = ncfile.createVariable("'+vecvar+'(::)","f8",("vec","nzetaFull_axis"))')

    exec(vecval+'_var[:,:] = '+vecval)

ncfile.title = "== File that containts axis and boundary information, used in GVEC with the hmap_axisNB module"
hdr=  "======= HEADER OF THE NETCDF FILE VERSION 3.0 ==================================="
hdr+= "\n    Note: This file was generated from pyQSC data."
hdr+= "\n=== FILE DESCRIPTION:"
hdr+= "\n  * axis, normal and binormal of the frame are given in cartesian coordinates along the curve parameter zeta [0,2pi]."
hdr+= "\n  * The curve is allowed to have a field periodicity NFP, but the curve must be provided on a full turn."
hdr+= "\n  * The adata is given in real space, sampled along equidistant zeta point positions:"
hdr+= "\n      zeta(i)=(i+0.5)/nzeta * (2pi/NFP), i=0,...,nzeta-1"
hdr+= "\n    always shifted by (2pi/NFP) for the next field period."
hdr+= "\n    Thus the number of points along the axis for a full turn is NFP*nzeta"
hdr+= "\n  * definition of the axis-following frame in cartesian coordinates ( boundary surface at rho=1):"
hdr+= "\n"
hdr+= "\n     {x,y,z}(rho,theta,zeta)=axis_{x,y,z}(zeta) + X(rho,theta,zeta)*N_{x,y,z}(zeta)+Y(rho,theta,zeta)*B_{x,y,z}(zeta)  "
hdr+= "\n"
hdr+= "\n=== DATA DESCRIPTION"
hdr+= "\n- general data"
hdr+= "\n  * NFP: number of field periods"
hdr+= "\n  * VERSION: version number as integer: V3.0 => 300"
hdr+= "\n- 'axis' data group:"
hdr+= "\n  * 'axis/nzeta'   : number of points along the axis, in one field period (>=2*n_max+1)"
hdr+= "\n  * 'axis/zeta(:)' : zeta positions, 1D array of size 'nzeta',  must exclude the end point, on half grid for one field period! zeta(i)=(i+0.5)/nzeta*(2pi/nfp), i=0,...nzeta-1"
hdr+= "\n  * 'axis/xyz(::)' : cartesian positions along the axis for ONE FULL TURN, 2D array of size (3,NFP* nzeta ), sampled at zeta positions," 
hdr+= "\n                     xyz[:,j+fp*nzeta]=axis(zeta[j]+fp*2pi/NFP), for j=0,..nzeta-1 and  fp=0,...,NFP-1"
hdr+= "\n  * 'axis/Nxyz(::)': cartesian components of the normal vector of the axis frame, 2D array of size (3, NFP* nzeta), evaluated analogously to the axis"
hdr+= "\n  * 'axis/Bxyz(::)': cartesian components of the bi-normal vector of the axis frame, 2D array of size (3, NFP*nzeta), evaluated analogously to the axis"
hdr+= "\n- 'boundary' data group:"
hdr+= "\n  * 'boundary/m_max'    : maximum mode number in theta "
hdr+= "\n  * 'boundary/n_max'    : maximum mode number in zeta (in one field period)"
hdr+= "\n  * 'boundary/lasym'    : asymmetry, logical. "
hdr+= "\n                           if lasym=0, boundary surface position X,Y in the N-B plane of the axis frame can be represented only with"
hdr+= "\n                             X(theta,zeta)=sum X_mn*cos(m*theta-n*NFP*zeta), with {m=0,n=0...n_max},{m=1...m_max,n=-n_max...n_max}"
hdr+= "\n                             Y(theta,zeta)=sum Y_mn*sin(m*theta-n*NFP*zeta), with {m=0,n=1...n_max},{m=1...m_max,n=-n_max...n_max}"
hdr+= "\n                           if lasym=1, full fourier series is taken for X,Y"
hdr+= "\n  * 'boundary/ntheta'    : number of points in theta (>=2*m_max+1)"
hdr+= "\n  * 'boundary/nzeta'     : number of points in zeta  (>=2*n_max+1), can be different to 'axis/nzeta' !"
hdr+= "\n  * 'boundary/theta(:)'  : theta positions, 1D array of size 'ntheta', on half grid! theta(i)=(i+0.5)/ntheta*(2pi), i=0,...ntheta-1"
hdr+= "\n  * 'boundary/zeta(:)'   : zeta positions , 1D array of size 'nzeta' , on half grid for one field period! zeta(i)=(i+0.5)/nzeta*(2pi/nfp), i=0,...nzeta-1"
hdr+= "\n  * 'boundary/X(::)',"
hdr+= "\n    'boundary/Y(::)'     : boundary position X,Y in the N-B plane of the axis frame, in one field period, 2D array of size(ntheta, nzeta),  with"
hdr+= "\n                              X[i, j]=X(theta[i],zeta[j])"
hdr+= "\n                              Y[i, j]=Y(theta[i],zeta[j]), i=0...ntheta-1,j=0...nzeta-1"

ncfile.header = hdr
ncfile.close()

In [ ]:
ncfile = nc.Dataset(fn+'.nc', 'r') 
for var in ["NFP","VERSION","axis/nzeta"]:
    print(var+"=",np.asarray(ncfile[var]).item())


print(ncfile.header)

In [ ]:
#change parametrization of curve to get a better fourier spectrum of the boundary


z=stel.phi/(2*np.pi/stel.nfp)
#plt.plot(z,stel.d_l_d_phi)

var=stel.d_l_d_phi
#var=1/stel.curvature**2 
l=0*var
for i in range(1,stel.phi.shape[0]):
    l[i]=l[i-1]+0.5*(var[i]+var[i-1])*(stel.phi[i]-stel.phi[i-1])
ltotal=l[-1]+0.5*(var[0]+var[-1])*(stel.phi[0]+(2*np.pi/stel.nfp)-stel.phi[-1])

l=stel.varphi
ltotal=2*np.pi/stel.nfp

fig = plt.figure(figsize=(18, 10))


# toroidal flux
ax = fig.add_subplot(2, 2, 1)
ax.plot(z,stel.X1c,label="X1c over phi")


#ax.plot(z,1/stel.curvature,label="1/curvature")
#ax.plot(z,stel.d_l_d_phi,label="lprime")

#ax.plot(z,l/ltotal-z,label="l")

ax.plot(l/ltotal,stel.X1c,label = "X1c over l")
#plt.plot(z,-0.7-0.8*0.5*(1-np.cos(2*np.pi*z)),label="cos")
ax.legend()

ax = fig.add_subplot(2, 2, 2)
ax.plot(z,stel.Y1c,label="Y1c over phi")
ax.plot(l/ltotal,stel.Y1c,label = "Y1c over l")
#plt.plot(z,np.sin(2*np.pi*z),label="sin")
ax.legend()

#stel.phi/(2*np.pi/stel.nfp)

In [ ]:
print(stel.nphi)
#np.sum(stel.torsion*stel.d_l_d_phi)/stel.nphi

#compute rotation angle l=theta=int(tau*lprime)
z=stel.phi/(2*np.pi/stel.nfp)
var=-stel.torsion*stel.d_l_d_phi
l=0*var
for i in range(1,stel.phi.shape[0]):
    l[i]=l[i-1]+0.5*(var[i]+var[i-1])*(stel.phi[i]-stel.phi[i-1])
ltotal=l[-1]+0.5*(var[0]+var[-1])*(stel.phi[0]+(2*np.pi/stel.nfp)-stel.phi[-1])

print(ltotal)
l=l-z*ltotal # make it periodic

#plt.plot(z,stel.torsion,label="-torsion over phi")
plt.plot(z,l/np.pi,label="-theta(tau)/pi over phi")
plt.plot(z,stel.X1c,label="X1c over phi")
#plt.plot(z,stel.Y1c,label="Y1c over phi")
#plt.plot(z,stel.Y1s,label="Y1s over phi")
plt.plot(z,stel.X1c*np.cos(l)-stel.X1s*np.sin(l),label="X1c' over phi")
plt.plot(z,stel.X1c*np.sin(l)+stel.X1s*np.cos(l),label="X1s' over phi")
#plt.plot(z,stel.Y1c*np.cos(l)-stel.Y1s*np.sin(l),label="Y1c' over phi")
#plt.plot(z,stel.Y1c*np.sin(l)+stel.Y1s*np.cos(l),label="Y1s' over phi")
plt.legend()

In [ ]:
from qsc.util import to_Fourier
def to_1D_Fourier(x,ntor):
    x_2D=np.ones((4,1))*x.T
    xc, xs, tmpc, tmps =to_Fourier(x_2D,x_2D,stel.nfp,0,ntor,True)  # cos(0*thet-ntor*nfp*zeta),sin(0*thet-ntor*nfp*zeta)
    return xc[ntor:,0], -xs[ntor:,0]

def to_1D_real(rc,rs,n_modes,phi):
    nphi=np.expand_dims(n_modes,axis=0)*np.expand_dims(phi,axis=1)
    return np.add.reduce(np.expand_dims(rc,axis=0)*np.cos(nphi)+np.expand_dims(rs,axis=0)*np.sin(nphi),axis=1)


ntor=stel.phi.shape[0]//2
print("exact ntor:",ntor)
for var in  ["X1c","X1s","Y1c","Y1s"]:
    print("check to Fourier and back: "+var)
    svar=eval("stel."+var)
    xc, xs =to_1D_Fourier(svar,ntor)
    n_modes=np.arange(0,ntor+1)*stel.nfp
    x_p = to_1D_real(xc,xs,n_modes,stel.phi)
    err=np.amax(np.abs(x_p-svar))
    if( not (np.allclose(x_p,svar,rtol=1e-08))):
        print("... failed : "+var)
        print("error",err)
    else:
        print("... sucessful","maxabs",err)
    min_ntor=ntor
    while (np.allclose(x_p,svar,rtol=1e-08) and (min_ntor > 0)):
        err=np.amax(np.abs(x_p-svar))
        min_ntor=min_ntor-1
        n_modes=np.arange(0,min_ntor+1)*stel.nfp
        x_p = to_1D_real(xc[0:min_ntor+1],xs[0:min_ntor+1],n_modes,stel.phi)
    print("found min_ntor %d with maxabs=%11.3e" %(min_ntor,err))
            


In [ ]:
stel.to_vmec(filename="input_test")

In [ ]:
### FIRST CONVERTER TO GVEC, for order "r1"


ntor=15
n_modes=np.arange(0,ntor+1)*stel.nfp
togvec={}
#togvec["r1"]={"xcvar":"X1c","xsvar":"X1s","ycvar":"Y1c","ysvar":"Y1s",
#              "xcvar_g":"X1_b_cos","xsvar_g":"X1_b_sin","ycvar_g":"X2_b_cos","ysvar_g":"X2_b_sin","m":1}
togvec={"gvec":{"xcvar":"X1_b_cos",
                "xsvar":"X1_b_sin",
                "ycvar":"X2_b_cos",
                "ysvar":"X2_b_sin"}
        , "r1":{"xcvar":"X1c_untwisted",
                "xsvar":"X1s_untwisted",
                "ycvar":"Y1c_untwisted",
                "ysvar":"Y1s_untwisted","m":1}
        , "r2":{"x0var":"X20_untwisted",
                "xcvar":"X2c_untwisted",
                "xsvar":"X2s_untwisted",
                "y0var":"Y20_untwisted",
                "ycvar":"Y2c_untwisted",
                "ysvar":"Y2s_untwisted","m":2}
        }

r=0.1
print("!---- hmap")
print("nfp        =",stel.nfp)
print("which_hmap = 20")
print("  hmap_nvisu = 25")
print("  hmap_n_max = ",stel.rc.shape[0]-1)
for var in ["rc","rs","zc","zs"]:
    svar=eval("stel."+var)
    if(any(np.abs(svar)>1.0e-12)):
        print('  hmap_'+var+'    = (/'+','.join(map(str, svar))+'/)')

print("!---- profiles")
print("sign_iota=1")
print("iota_coefs=(/ %23.11e /)" %(stel.iota))
print("pres_coefs=(/ %23.11e /)" %(0.0))
#print("!helicity=",stel.helicity)
#print("!iotaN=",stel.iotaN)

print("PHIEDGE   =",np.pi * r * r * stel.spsi * stel.Bbar)

r1=togvec["r1"]
r2=togvec["r2"]
m_max=1
th=np.linspace(0,2*np.pi,4*m_max,endpoint=False)
th=np.expand_dims(th,axis=1)

if(True):    
    m=1
    xc_p = eval("stel."+r1["xcvar"])
    xs_p = eval("stel."+r1["xsvar"])
    yc_p = eval("stel."+r1["ycvar"])
    ys_p = eval("stel."+r1["ysvar"])
    x_2d=r*(np.cos(m*th)*xc_p+np.sin(m*th)*xs_p)
    y_2d=r*(np.cos(m*th)*yc_p+np.sin(m*th)*ys_p)
    
    m=2
    if (m <= m_max):
        x0_p = eval("stel."+r2["x0var"])
        xc_p = eval("stel."+r2["xcvar"])
        xs_p = eval("stel."+r2["xsvar"])
        y0_p = eval("stel."+r2["y0var"])
        yc_p = eval("stel."+r2["ycvar"])
        ys_p = eval("stel."+r2["ysvar"])
        x_2d+=r*r*(x0_p+np.cos(m*th)*xc_p+np.sin(m*th)*xs_p)
        y_2d+=r*r*(y0_p+np.cos(m*th)*yc_p+np.sin(m*th)*ys_p)
        # Z NOT YET INCLUDED, WOULD NEED INTERPOLATION TO THE (N,B) plane!!
    
    # transform to cos(m*th-n*nfp*phi),sin(m*th-n*nfp*phi)
    xc,xs,yc,ys=to_Fourier(x_2d,y_2d,stel.nfp,m_max,ntor,True)
    
    for m in range(1,m_max+1):
        for var in ["xc","xs","yc","ys"]:
            var_g=togvec["gvec"][var+"var"]
            x=eval(var)
            if np.any((np.abs(x) >1.0e-12)) :
                print("!translate to gvec's %s(m,-ntor..ntor), r=%f, m=%d, ntor=%d" %(var_g,r,m,ntor))
            for n in range(-ntor,ntor+1):
                if (np.abs(x[n+ntor,m]) >1.0e-12):
                    print("%s(%2d;%2d) = %21.11e" %(var_g,m,n,x[n+ntor,m]))

### Convert boundary to GVEC RZ (theta,zeta)

Note that both angular directions must change sign to keep a positive Jacobian in GVEC

In [ ]:
from qsc.util import to_Fourier

r=0.1
ntheta=40
# Get surface shape at fixed off-axis toroidal angle phi
R_2D, Z_2D, phi0_2D = stel.Frenet_to_cylindrical(r, ntheta)
    
# Fourier transform the result.
# This is not a rate-limiting step, so for clarity of code, we don't bother with an FFT.
mpol=9
ntor=14
RBC, RBS, ZBC, ZBS = to_Fourier(R_2D, Z_2D, stel.nfp, mpol, ntor, stel.lasym)

eps=1.0e-12
#translate axis to GVEC-RZ (cos(m*theta-n*zeta),sin(m*theta-n*zeta)), with zeta=-phi:
# n>=0: rc_n*cos(n*phi)=rc_n*cos(-n*zeta) , rs_n*sin(n*phi)=rs_n*sin(-n*zeta)
print("nfp        =",stel.nfp)
print("!---- hmap RZ frame")
print("which_hmap=1")
print('! axis coefficients in RZ frame, only for initalization, ntor=%d'%(ntor))

if(stel.lasym):
    tvars=["rc","rs","zc","zs"]
    gvars=["X1_a_cos","X1_a_sin","X2_a_cos","X2_a_sin"]
else:
    tvars=["rc","zs"]
    gvars=["X1_a_cos","X2_a_sin"]
ntor=stel.rc.shape[0]-1
for var,gvar  in zip(tvars,gvars):
    evar2=eval("stel."+var)
    m=0
    for n in range(0,ntor+1):
        if(np.abs(evar2[n])>eps):
            print('%s( %3d; %3d)=%21.11e' %(gvar,m,n,evar2[n]))

            
print('! boundary coefficients in RZ frame at r=%f, lasym=%s,mpol=%d,ntor=%d'%(r,str(stel.lasym),mpol,ntor))

if(stel.lasym):
    tvars=["RBC","RBS","ZBC","ZBS"]
    gvars=["X1_b_cos","X1_b_sin","X2_b_cos","X2_b_sin"]
    sgns=[1.0,-1.0,1.0,-1.0]
else:
    tvars=["RBC","ZBS"]
    gvars=["X1_b_cos","X2_b_sin"]
    sgns=[1.0,-1.0]

#translate to GVEC-RZ (cos(m*theta-n*zeta),sin(m*theta-n*zeta)) with zeta=-phi and theta_g=-theta:
# m=0 n>=0: RBC_n*cos(m*theta-n*phi)=RBC_n*cos(m*theta_g-n*zeta) , RBS_n*sin(m*theta-n*phi)=-RBS_n*sin(m*theta_g-n*zeta)
# for m>0,n=ntor...ntor, n*phi = -n*zeta, so (n) -> (-n)
mpol=eval(tvars[0]).shape[1]-1
ntor=(eval(tvars[0]).shape[0]-1)//2
for tvar,gvar,sgn  in zip(tvars,gvars,sgns):    
    evar=eval(tvar)
    m=0
    for n in range(0,ntor+1):
        if(np.abs(evar[n+ntor,m])>eps):
            print('%s( %3d; %3d)=%21.11e' %(gvar,m,n,sgn*evar[n+ntor,m])) # sign change for sin(-n*(phi=-zeta))=-sin(-n*zeta), since n>=0
    for m in range(1,mpol+1):
        for n in range(-ntor,ntor+1):
            if (np.abs(evar[n+ntor,m])>eps):
                #print('%s( %3d; %3d)=%21.11e' %(gvar,m,-n,evar[n+ntor,m])) # sign change of zeta by switching n-> -n
                print('%s( %3d; %3d)=%21.11e' %(gvar,m,n,sgn*evar[n+ntor,m])) # sign change of zeta  and theta!


In [ ]:
from qsc.Frenet_to_Frenet_NB import Frenet_to_Frenet_NB
from qsc.util import to_Fourier

r=0.1
ntheta=40
# Get surface shape at all phi axis position in the NB plane
Xhat_2D, Yhat_2D, phi0_2D =  Frenet_to_Frenet_NB(stel,r, ntheta)
    
# Fourier transform the result.
# This is not a rate-limiting step, so for clarity of code, we don't bother with an FFT.
mpol=10
ntor=15
XBC, XBS, YBC, YBS = to_Fourier(Xhat_2D, Yhat_2D, stel.nfp, mpol, ntor, stel.lasym)

eps=1.0e-12

print("!---- hmap frenet")
print("nfp        =",stel.nfp)
print("which_hmap = 20")
print("  hmap_nvisu = 25")
print("  hmap_n_max = ",stel.rc.shape[0]-1)
for var in ["rc","rs","zc","zs"]:
    svar=eval("stel."+var)
    if(any(np.abs(svar)>1.0e-12)):
        print('  hmap_'+var+'    = (/'+','.join(map(str, svar))+'/)')


if(stel.lasym):
    tvars=["XBC","XBS","YBC","YBS"]
    gvars=["X1_b_cos","X1_b_sin","X2_b_cos","X2_b_sin"]
else:
    tvars=["XBC","YBS"]
    gvars=["X1_b_cos","X2_b_sin"]
#translate to GVEC-FRENET (cos(m*theta-n*zeta),sin(m*theta-n*zeta)) with zeta=-phi and theta_g=-theta:
# m=0 n>=0: RBC_n*cos(m*theta-n*phi)=RBC_n*cos(m*theta_g-n*zeta) , RBS_n*sin(m*theta-n*phi)=-RBS_n*sin(m*theta_g-n*zeta)
# for m>0,n=ntor...ntor, n*phi = -n*zeta, so (n) -> (-n)

print('! boundary coefficients in FRENET FRAME at r=%f, lasym=%s,mpol=%d,ntor=%d'%(r,str(stel.lasym),mpol,ntor))
for tvar,gvar,sgn  in zip(tvars,gvars,sgns):    
    evar=eval(tvar)
    m=0
    for n in range(0,ntor+1):
        if(np.abs(evar[n+ntor,m])>eps):
            print('%s( %3d; %3d)=%21.11e' %(gvar,m,n,sgn*evar[n+ntor,m])) # sign change for sin(-n*(phi=-zeta))=-sin(-n*zeta), since n>=0
    for m in range(1,mpol+1):
        for n in range(-ntor,ntor+1):
            if (np.abs(evar[n+ntor,m])>eps):
                #print('%s( %3d; %3d)=%21.11e' %(gvar,m,-n,evar[n+ntor,m])) # sign change of zeta by switching n-> -n
                print('%s( %3d; %3d)=%21.11e' %(gvar,m,n,sgn*evar[n+ntor,m])) # sign change of zeta  and theta!
